# Path
1. Create the model in GCP
    - Create the Dialogflow structure
    - Implement the PaLM into the Dialogflow
    - Test it out 
2. Make the backend locally
3. Create a Front End
4. Deploy the Front End

## Future Improvements
- Add identification in the frontend and change the conversational model if the user is identified. For example in the welcome intent instead of saying the fir interaction, get it ready to do something

---

# Imports
Importing all the libraries needed to run the notebook

In [2]:
from google.cloud import dialogflow
import uuid
from google.oauth2 import service_account
import os
from dotenv import load_dotenv
import gradio as gr
import random
import time
from PIL import Image
import numpy as np

c:\Users\borga\Desktop\IE Computer Science\Semester 3\NLP Conversational Model\Final Project\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load the environment variables
load_dotenv()

True

# Agent creation
---
**ADD IMAGES ONCE FINISHED**


# Agent connection
---
In order to connect to the project created in GCP and tHe agent created in Dialogflow we will have to follow a series of steps starting from authentication
### Authentication
Authentication as a service account in the Google Cloud Platform to access the resources in the cloud

In [4]:
# Authenticate the computer as a service account
service_account_key = os.getenv('service_account_key')
credentials = service_account.Credentials.from_service_account_file(service_account_key)

### Session Creation
A session represents a conversation between a Dialogflow agent and an end-user. You create a session at the beginning of a conversation and use it for each turn of the conversation. Once the conversation has ended, you discontinue using the session

In [5]:
session_id = uuid.uuid4().hex

In [6]:
# Load the variables to access the databases
project_id = os.getenv('project_id')
language_code = 'en'

In [7]:
# Create the session with the appropriate credentials
session_client = dialogflow.SessionsClient(credentials=credentials)

session = session_client.session_path(project_id, session_id)

### Agent function
Function that given an input returns the output of by connecting to the agent

In [8]:
# Request an answer to our dialogflow agent according to a text
def dialogflow_request(text, language_code):
    text_input = dialogflow.TextInput(text=text, language_code=language_code)
    query_input = dialogflow.QueryInput(text=text_input)
    response = session_client.detect_intent(
        request={"session": session, "query_input": query_input}
    )
    return response

# Web Application 
---
Web application using Gradio to interact with the agent

### Handle Images
The user will hav the opportunity of inputting images to the chatbot. In order to do so the following functions are needed

In [54]:
# Function to get an answer from Dialogflow according to an input
def respond(message, chat_history):
    # Use the dialogflow function to get the answer
    response = dialogflow_request(message, 'en')
    bot_message = str(response.query_result.fulfillment_messages[0].text.text[0])
    chat_history.append((message, bot_message))
    time.sleep(1)
    return "", chat_history

# Function to add text to the chat
def add_text(history, text):
    history.append((text, None))
    return history, respond(text)

# Function to add a file
def add_file(history, file):
    history.append(((file.name,), None))
    return history

In [55]:
# Gradio Web App
with gr.Blocks() as demo:
    # Create a chatbot component
    chatbot = gr.Chatbot([], elem_id="chatbot", label="FitBot").style()
    with gr.Row():
        # Create a textbox component
        with gr.Column(scale=0.85):
            msg = gr.Textbox(
                show_label=False,
                placeholder="Enter text and press enter, or upload an image",
            ).style(container=False)
        with gr.Column(scale=0.15, min_width=0):
            # Create an upload button component to add images
            btn = gr.UploadButton("📁", file_types=["image"])
    clear_btn = gr.Button(value="🗑️  Clear")
    
    # Call the functions previously created
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    file_msg = btn.upload(add_file, [chatbot, btn], [chatbot], queue=False)
    clear_btn.click(lambda: None, None, chatbot, queue=False)

# launch the application
demo.launch()

c:\Users\borga\Desktop\IE Computer Science\Semester 3\NLP Conversational Model\Final Project\myenv\lib\site-packages\gradio\components\chatbot.py:228: UserWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  warnings.warn(
c:\Users\borga\Desktop\IE Computer Science\Semester 3\NLP Conversational Model\Final Project\myenv\lib\site-packages\gradio\components\textbox.py:259: UserWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7887

To create a public link, set `share=True` in `launch()`.


# LLM

In [13]:
from vertexai.preview.language_models import TextGenerationModel
from google.cloud import aiplatform


In [14]:
region = os.getenv('region')

In [7]:
aiplatform.init(project=project_id, location=region)

In [17]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = service_account_key

In [21]:
# Test of text model
parameters = {
    "temperature": 0.2,  # Temperature controls the degree of randomness in token selection.
    "max_output_tokens": 1024,  # Token limit determines the maximum amount of text output.
    "top_p": 0.8,  # Tokens are selected from most probable to least until the sum of their probabilities equals the top_p value.
    "top_k": 40,  # A top_k of 1 means the selected token is the most probable among all tokens.
}

model = TextGenerationModel.from_pretrained("text-bison@001")
response = model.predict('What are nutrients?', **parameters)
print(f"Response from Model: {response.text}")
##print(f"Raw Response: {response}")
# Create the Vertex AI client
client = aiplatform.gapic.PredictionServiceClient()

Response from Model: Nutrients are substances that provide the body with energy and the materials it needs to grow, develop, and repair itself. Nutrients are found in food and beverages.
Endpoint URL: aiplatform.googleapis.com:443
